# Julia Multiple Dispatch (多重分派)

## 1. Multiple Dispatch (多重分派)

多重分派被認為是 Julia 語言中最強大的單一特性 (在官方文件中指出: "... multiple dispatch on the types of values is perhaps the single most powerful and central feature of the Julia language.")

多重分派的觸發時機是在執行期間 (runtime), 呼叫函式時根據傳入引數 (argument) 的數目及型別, 判斷應該呼叫的方法 (method). 之所以稱之為 **multiple** dispatch 的原因是, Julia 判斷時是根據所有引數, 而非像在某些程式語言中是根據第一個引數做判斷.

與多重分派相似的程式語言概念是 _Function Overloading_, 最大的不同點在於, Function Overloading 的觸發時機點在編譯 (compile time) 期間.

### 1.1 Methods

Julia 內建的函式均已使用多重分派實作, 例如下面的例子, 使用 `methods()` 函式列出加法函式的實作定義, 可以看到因多重分派, 我們可以傳入不同型別的值到加法函式中進行運算.

Method 是 Function (函式) 的行為 (behavior) 定義, 一個函式可以被定義多個行為, 也就是多個 Method, 在 Method 中再去進行不同的實作.

In [0]:
methods(+)

# 166 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:282
[2] +(x::Bool, y::Bool) in Base at bool.jl:96
[3] +(x::Bool) in Base at bool.jl:93
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:104
[5] +(x::Bool, z::Complex) in Base at complex.jl:289
[6] +(a::Float16, b::Float16) in Base at float.jl:398
[7] +(x::Float32, y::Float32) in Base at float.jl:400
[8] +(x::Float64, y::Float64) in Base at float.jl:401
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:283
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:297
[11] +(::Missing, ::Missing) in Base at missing.jl:114
[12] +(::Missing) in Base at missing.jl:100
[13] +(::Missing, ::Number) in Base at missing.jl:115
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:106
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:413
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:503
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:502
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:501
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:472
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt8}) in Base.GMP at gmp.jl:509
[21] +(x::BigInt, c::Union{Int16, Int32, Int8}) in Base.GMP at gmp.jl:515
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:563
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:556
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:550
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:409
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:378
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt8}) in Base.MPFR at mpfr.jl:385
[28] +(x::BigFloat, c::Union{Int16, Int32, Int8}) in Base.MPFR at mpfr.jl:393
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:401
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\Dates\src\periods.jl:343
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\Dates\src\periods.jl:341
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\Dates\src\periods.jl:371
[33] +(x::Dates.Date, y::Dates.Day) in Dates at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\Dates\src\arithmetic.jl:74
[34] +(x::Dates.Date, y::Dates.Week) in Dates at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\Dates\src\arithmetic.jl:72
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\Dates\src\arithmetic.jl:55
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\Dates\src\arithmetic.jl:28
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\Dates\src\arithmetic.jl:19
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\Dates\src\arithmetic.jl:20
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\Dates\src\arithmetic.jl:78
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\Dates\src\arithmetic.jl:48
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\Dates\src\arithmetic.jl:24
[42] +(x::Dates.DateTime, y::Dates.Period) in Dates at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\Dates\src\arithmetic.jl:76
[43] +(B::BitArray{2}, J::LinearAlgebra.UniformScaling) in LinearAlgebra at D:\buildbot\worker\package_win64\build\usr\share

若未透過 `::` 定義型別的話, 則是 `Any` 型別.

In [0]:
f(x::Number, y::Number) = 2x - 3y
f(x::Float64, y::Float64) = 2x - 3y
f(x, y) = println("Dispatched to Any")

f (generic function with 3 methods)

呼叫 `methods()` 函式可以顯示 Function 中所有的 Method 定義, 以及其行數.

In [0]:
methods(f)

# 3 methods for generic function "f":
[1] f(x::Float64, y::Float64) in Main at In[2]:2
[2] f(x::Number, y::Number) in Main at In[2]:1
[3] f(x, y) in Main at In[2]:3

根據數目和傳入值的型別, Multiple Dispatch 會判斷應該要分派到哪一個 method. 多重分派會優先尋找最符合的定義來分派.

In [0]:
# 符合 Float64
f(5.0, 3.0)

1.0

In [0]:
# 最符合 Number
f(5, 3)

1

In [0]:
# 沒有定義 String 的 Method, 所以最符合的是 Any
f("5", "3")

Dispatched to Any


透過 `@which` macro 巨集, 可以看到會被分派到哪一個 method.

In [0]:
@which f(5.0, 3.0)

f(x::Float64, y::Float64) in Main at In[2]:2

含糊 (ambiguous) 的定義可能會造成 `MethodError`.

In [0]:
f2(x::Float64, y) = 2x - 3y
f2(x, y::Float64) = 2x - 3y

f2(2.0, 3.0)

MethodError: MethodError: f2(::Float64, ::Float64) is ambiguous. Candidates:
  f2(x::Float64, y) in Main at In[8]:1
  f2(x, y::Float64) in Main at In[8]:2
Possible fix, define
  f2(::Float64, ::Float64)

### 1.2 型別參數化 (Type Parameter) 的 Method (Parametric Methods)

Method 可以使用 **type** 參數做為引數的型別, 如下例示範, 傳入引數要屬於 Number 或是其子型別, 且 2 個引數型別相同.

In [0]:
f3(x::T, y::T) where{T<:Number} = 2x - 3y

f3 (generic function with 1 method)

In [0]:
f3(5, 3)

1

In [0]:
# 傳入引數型別不同, 產生 MethodError
f3(5, 3.0)

MethodError: MethodError: no method matching f3(::Int64, ::Float64)
Closest candidates are:
  f3(::T, !Matched::T) where T<:Number at In[9]:1

In [0]:
# String 不屬於 Number 型別及其子型別, 所以會產生 MethodError
f3("123", "456")

MethodError: MethodError: no method matching f3(::String, ::String)